In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv('fashion-mnist_train.csv')
data.head()


,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) 

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [4]:
Y_train

array([9, 5, 1, ..., 5, 3, 5], dtype=int64)

In [5]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [6]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [7]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 6000)


Iteration:  0
[5 5 5 ... 5 5 5] [9 5 1 ... 5 3 5]
0.08649152542372882
Iteration:  10
[9 9 1 ... 5 1 0] [9 5 1 ... 5 3 5]
0.3062542372881356
Iteration:  20
[9 9 1 ... 5 1 9] [9 5 1 ... 5 3 5]
0.31357627118644066
Iteration:  30
[9 9 1 ... 5 1 9] [9 5 1 ... 5 3 5]
0.33576271186440676
Iteration:  40
[9 9 1 ... 5 6 9] [9 5 1 ... 5 3 5]
0.3496271186440678
Iteration:  50
[9 9 1 ... 5 6 9] [9 5 1 ... 5 3 5]
0.3752542372881356
Iteration:  60
[9 9 1 ... 5 6 9] [9 5 1 ... 5 3 5]
0.401135593220339
Iteration:  70
[9 9 1 ... 5 6 9] [9 5 1 ... 5 3 5]
0.423135593220339
Iteration:  80
[9 9 1 ... 5 6 8] [9 5 1 ... 5 3 5]
0.4452372881355932
Iteration:  90
[9 9 1 ... 5 6 8] [9 5 1 ... 5 3 5]
0.4749322033898305
Iteration:  100
[7 9 1 ... 5 6 8] [9 5 1 ... 5 3 5]
0.509542372881356
Iteration:  110
[7 9 1 ... 5 6 8] [9 5 1 ... 5 3 5]
0.5433220338983051
Iteration:  120
[9 9 1 ... 5 6 8] [9 5 1 ... 5 3 5]
0.5659322033898305
Iteration:  130
[9 9 1 ... 5 6 8] [9 5 1 ... 5 3 5]
0.5818474576271186
Iteration:  140
[